In [ ]:
# %pip install pandas pyarrow
# %pip install tqdm
# %pip install plotly
# %pip install nbformat
# %pip install ipywidgets
# %pip install numba

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import os

# Imports from local scripts.
from data_downloader import download, get_filename
from trailing_stop_loss import generate_stop_loss_levels
from trailing_stop_loss import calculate_pl

In [ ]:
ticker = 'AAPL'
interval = '5s'
year = '2024'

In [ ]:
# get data

filename = get_filename(ticker, interval, year)
# If file doesn't exist, then download it.
if not os.path.exists(filename):
	download(ticker, interval, year)

df = pd.read_parquet(filename)

# # show what is loaded
# fig = px.line(df[-1000:], y='open', title=f'{ticker} Open Prices')
# fig.show()

total_bars = df.shape[0]
print(f'Total bars: {total_bars} ({total_bars:,})')

In [ ]:
# Set trailing stop-loss levels
max_sl = 1
sl_steps = 10
stop_loss_percents, stop_loss_levels = generate_stop_loss_levels(max_sl, sl_steps)

print("Stop-loss levels (in percent):")
print(stop_loss_percents)
print("\nStop-loss levels (as fractions):")
print(stop_loss_levels)

In [ ]:
results_df = calculate_pl(df, stop_loss_levels)
# results_df

In [ ]:
# Step 4: Visualization of results
import plotly.graph_objects as go

def plot_sl_profits(df_exp, results_df):
    fig = go.Figure()

    # Add stop-loss points
    # For better performance, we can add all points in one call
    # Filter out zero SL, as they do not carry information
    plot_df = results_df.copy()#[results_df['sl_level'] != 0].copy()
    plot_df['sl_price'] = plot_df['entry_price'] * (1 + plot_df['sl_level'])

    min_profit = plot_df['profit'].min()
    max_profit = plot_df['profit'].max()

    # Calculate normalized position for zero (between 0 and 1)
    zero_position = (0 - min_profit) / (max_profit - min_profit)

    custom_colorscale = [
        [0.0, 'red'],           # Minimum profit = red
        [zero_position, 'white'], # Zero profit = white
        [1.0, 'blue']           # Maximum profit = blue
    ]

    fig.add_trace(go.Scatter(
        x=plot_df['index'],
        y=plot_df['sl_price'],
        mode='markers',
        marker=dict(
            size=3,
            color=plot_df['profit'],
            colorscale=custom_colorscale,
            showscale=True,
            colorbar=dict(title='Profit (%)'),
            cmin=min_profit,  # Set the actual data range
            cmax=max_profit
        )
    ))

    # Add OHLC prices on top
    fig.add_trace(go.Scatter(
        # x=df_exp['timestamp'],
        y=df_exp['open'],
        mode='lines',
        line=dict(color='black', width=1)
    ))
    fig.add_trace(go.Scatter(
        y=df_exp['high'],
        mode='lines',
        line=dict(color='black', width=1)
    ))
    fig.add_trace(go.Scatter(
        y=df_exp['low'],
        mode='lines',
        line=dict(color='black', width=1)
    ))

    fig.update_layout(
        title="Trailing SL profits",
        showlegend=False
    )

    fig.show()

In [ ]:
# Import result slicing utilities
from trailing_stop_loss import slice_results_by_time

In [ ]:
plot_shift = 75*1000
plot_window = 10*1000
plot_df = slice_results_by_time(results_df, start_idx=plot_shift, num_points=plot_window)
plot_sl_profits(df[plot_shift:plot_shift + plot_window], plot_df)

In [ ]:
plot_shift = 85*1000
plot_window = 10*1000
plot_df = slice_results_by_time(results_df, start_idx=plot_shift, num_points=plot_window)
plot_sl_profits(df[plot_shift:plot_shift + plot_window], plot_df)

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Custom plot function showing specified percentiles and the mean for each category
def box(df, x, y, title=None, percentiles=[0, 10, 25, 50, 75, 90, 100]):
    """
    Custom box-like plot function showing specified percentiles and the average for each category.
    
    Parameters:
    df         : pandas.DataFrame
    x          : str, column name for categories
    y          : str, column name for values
    title      : str, plot title
    percentiles: list of ints, percentiles to display (e.g., [0,10,25,50,75,90,100])
    """
    # Ensure percentiles are sorted and converted to fractions
    percentiles_sorted = sorted(percentiles)
    quantile_fracs = [p / 100 for p in percentiles_sorted]

    # Compute the specified percentile values for each group
    grouped_q = df.groupby(x)[y].quantile(quantile_fracs).unstack()
    # Compute the mean value for each group
    grouped_mean = df.groupby(x)[y].mean()

    # Initialize figure
    fig = go.Figure()

    # Add a trace for each percentile
    for p, frac in zip(percentiles_sorted, quantile_fracs):
        fig.add_trace(go.Scatter(
            x=grouped_q.index,
            y=grouped_q[frac],
            mode='lines+markers',
            name=f'{p}th percentile',
            line=dict(dash='solid' if p == 50 else 'dash'),
            marker=dict(size=6)
        ))

    # Add mean as a bold black line
    fig.add_trace(go.Scatter(
        x=grouped_mean.index,
        y=grouped_mean.values,
        mode='lines',
        name='Mean',
        line=dict(color='black', width=4),
        hovertemplate='Mean: %{y:.2f}<extra></extra>'
    ))

    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title=x,
        yaxis_title=y,
        legend_title='Metrics',
        template='plotly_white'
    )

    return fig

# Example usage:
# fig = box(results_df, x='sl_level', y='profit',
#           title='Profit Distribution by Stop-Loss Level',
#           percentiles=[0, 10, 25, 50, 75, 90, 100])
# fig.show()


In [ ]:
# Example usage
fig = box(results_df, x='sl_level', y='profit',
    title='Profit Distribution by Stop-Loss Level',
    percentiles=[0, 25, 50, 75, 90, 100])
fig.show()
